In [4]:
import warnings
warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload

import os

import pandas as pd

from sklearn.ensemble import RandomForestClassifier

from config.config import TOTAL_DAYS, PATH_TO_DATA
# рассматриваем данные за первые TOTAL_DAYS дней активности
from utils.df_creation import get_users, create_interaction, get_target, get_min_timestamps,\
    cut_df_by_time, cut_dfs_by_time, get_action_counts_by_period, get_base_features

Получим бейзлайн решение на базовых признаках, полученных в прошлой тетрадке.

In [5]:
basic_train_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_data_base.csv'))
basic_test_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'test_data_base.csv'))

In [6]:
basic_train_df.head()

,user_id,discovered,passed,started_attempt,viewed,correct,wrong,num_days,passed_course
0,1,1,0,0,1,0,0,1,0
1,2,9,9,2,9,2,0,1,0
2,3,15,15,4,20,4,4,1,0
3,5,1,1,0,1,0,0,1,0
4,7,1,1,0,1,0,0,1,0


In [ ]:
rf = RandomForestClassifier(
    n_estimators=100,
    min_samples_leaf=10,
    min_samples_split=10,
    class_weight='balanced',
    n_jobs=-2
)

rf.fit(basic_train_df[:, ])

Можно выделить два (пересекающихся) типа поведения в первые два дня, при которых можно сделать вывод, что участник курса покидает его:

1. Пришел на курс, быстро понял, что "не его", и ушел. Это можно описать, например, следующими признаками:
- Число уникальных дней активности
- Большая разница между активностями в первый и во второй день
- Общее число часов, которые пользователь провел на курсе (hour_evn в текущем воплощении сделана неправильно, надо переделать)


2. Пришел на курс, понял, что не получается, и ушел. Это можно описать, например, следующими признаками:
- Не получилось решить какие-то конкретные степы (например, самые сложные или те, на которых заканчивает участвовать в курсе много людей)
- Число неудачных решений задач (```started_attempt - correct```)
- Число просмотренных, но не пройденных задач (```evn_steps - passed```)
- Отношение числа пройденных степов к общему числу степов
- Процент удачных решений


In [ ]:
def get_time_features(events, submissions):
    events['hour'] = events.time.dt.floor('H')
    events_tf = events.groupby('user_id').agg({
        'hour': 'nunique' # количество уникальных часов в events
    }).rename(columns={'hour': 'uniq_hour_evn'})

    submissions_tf = submissions_df.groupby('user_id').agg({
        'hour': 'nunique' # количество уникальных часов в submissions
    }).rename(columns={'hour': 'uniq_hour_sub'})
    
    time_features_df = events_tf.merge(submissions_tf, on='user_id', how='outer').fillna(-1)
    assert time_features_df.user_id.nunique() == events_df.user_id.nunique()
    return time_features_df


def get_step_count(df, name):
    step_count = df.groupby('user_id', as_index=False)[['step_id']].nunique().rename(columns={'step_id': name})
    
    return step_count
    
    
def get_steps_ohe_features(step_ids, submissions):
    
    # прошел ли пользователь степы из step_ids
    ohe_step = pd.get_dummies(
        submissions[(submissions.submission_status == 'correct') 
                       & (submissions.step_id.isin(step_ids))], 
        columns=['step_id']
    )
    steps_features = ohe_step.groupby('user_id').sum().reset_index()
    
    # отбор колонок с информацие о прохождении степов
    steps_features.rename(columns={'user_id': 'step_id_user_id'}, inplace=True)
    steps_features = steps_features.loc[:,steps_features.columns.str.startswith('step_id_')]
    steps_features.rename(columns={'step_id_user_id': 'user_id'}, inplace=True)
    
    return steps_features
    

def get_custom_features(df):
    # отобранные фичи
    df['dis_to_cor'] = df.discovered - df.correct # ???
    df['loss_step'] = df.started_attempt - df.correct # число неверных решений степов (на самом деле нет)
    df['step_pas'] = df.evn_steps - df.passed # число просмотренных, но не пройденных степов (на самом деле нет)
    df['start_pas'] = df.started_attempt - df.passed # ???
    df['all_sum'] = df[['correct', 'wrong', 'discovered', 'passed', 'started_attempt', 'viewed']].sum(axis=1)
    df['all_pass'] = df.all_sum - df.evn_steps
    
    return df

In [ ]:
def create_work_df(events, submissions, target=None):
    '''
    Сборка датасета для работы с моделью
    '''
    actions = create_interaction(events, submissions)
    df = get_base_features(actions)
    
    time_features = get_time_features(events, submissions)
    df = df.merge(time_features, on='user_id')
    df.rename(columns={
        ('hour_evn', ''): 'hour_evn', 
        ('day', ''): 'day',
        ('dayweek_evn', ''): 'dayweek_evn',  
        ('hour_sub', ''): 'hour_sub'
    }, inplace=True)
    
    df = df.merge(get_step_count(submissions, 'sub_steps'), on='user_id', how='outer').fillna(0)
    df = df.merge(get_step_count(events, 'evn_steps'), on='user_id')
    df = df.merge(get_steps_ohe_features(submissions), on='user_id', how='outer').fillna(0)

    df = get_custom_features(df)
    
    if target is not None:
        # добавление целевой переменной
        df = df.merge(target, on='user_id', how='outer').fillna(0)

    return df.astype('int')